## AI Workflow – Traditional vs. Structured Approach

### What This Notebook Covers
- **Manually setting up AI agents** (Traditional Approach)
- **Using a structured AI runtime** for automation (KitchenAI Approach)
- **Key benefits of structured AI workflows**


In [1]:
%pip install whisk llama-index toolhouse

  Using cached toolhouse-1.3.1-py3-none-any.whl.metadata (23 kB)
  Using cached anthropic-0.45.2-py3-none-any.whl.metadata (23 kB)
  Using cached groq-0.18.0-py3-none-any.whl.metadata (14 kB)
  Using cached http_exceptions-0.2.10-py3-none-any.whl.metadata (7.0 kB)
Using cached toolhouse-1.3.1-py3-none-any.whl (43 kB)
Using cached anthropic-0.45.2-py3-none-any.whl (222 kB)
Using cached groq-0.18.0-py3-none-any.whl (121 kB)
Using cached http_exceptions-0.2.10-py3-none-any.whl (8.8 kB)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import logging
import asyncio
from whisk.client import WhiskClient
from llama_index.llms.groq import Groq
from llama_index.core.agent import ReActAgent
from llama_index.core.memory import ChatMemoryBuffer
from toolhouse import Toolhouse, Provider
from whisk.kitchenai_sdk.kitchenai import KitchenAIApp
from whisk.kitchenai_sdk.schema import (
    WhiskQuerySchema,
    WhiskQueryBaseResponseSchema,
)
from whisk.kitchenai_sdk.schema import ChatInput
import uuid
import time


In [25]:

# Initialize logging
logger = logging.getLogger(__name__)

# Initialize LLM
llm = Groq(model="llama-3.2-11b-vision-preview")

# Initialize Toolhouse Instances for Different Tasks
scraper_toolhouse = Toolhouse(provider=Provider.LLAMAINDEX)
web_search_toolhouse = Toolhouse(provider=Provider.LLAMAINDEX)


# Set metadata (Optional)
scraper_toolhouse.set_metadata("id", "scraper")
web_search_toolhouse.set_metadata("id", "web_search")


# ✅ **Initialize KitchenAI App**
kitchen = KitchenAIApp(namespace="AI-Toolkit-Demo-2")

In [ ]:
agent = ReActAgent(
    tools=web_search_toolhouse.get_tools(bundle="web_search"),
    llm=llm,
    memory=ChatMemoryBuffer.from_defaults(),
)
response = await agent.achat(data.query)


Explanation:

We manually create a web search agent.
We attach the tool (web_search).
We call the LLM manually (achat()).
This works, but we have to repeat the setup for every agent.

In [ ]:
    agent = ReActAgent(
        tools=web_search_toolhouse.get_tools(bundle="scraper"),
        llm=llm,
        memory=ChatMemoryBuffer.from_defaults(),
    )
    response = await agent.achat(data.query)

📌 **Downsides of this approach:**  
❌ **Repetitive:** Need separate setups for every agent.  
❌ **Manual API Calls:** Every new capability requires more boilerplate.  
❌ **Hard to Scale:** If we add new AI tools, we must modify our code significantly.  

---

In [26]:



# ✅ **Quick AI Handlers**
@kitchen.query.handler("scraper")
async def quick_summary_handler(data: WhiskQuerySchema) -> WhiskQueryBaseResponseSchema:
    """Summarizes documents using doc_parser."""
    agent = ReActAgent(
        tools=scraper_toolhouse.get_tools(bundle="scraper"),
        llm=llm,
        memory=ChatMemoryBuffer.from_defaults(),
    )
    response = await agent.achat(data.query)
    return WhiskQueryBaseResponseSchema.from_llm_invoke(data.query, response.response)

@kitchen.query.handler("web-search")
async def web_search_handler(data: WhiskQuerySchema) -> WhiskQueryBaseResponseSchema:
    """Performs an AI-powered web search."""
    agent = ReActAgent(
        tools=web_search_toolhouse.get_tools(bundle="web_search"),
        llm=llm,
        memory=ChatMemoryBuffer.from_defaults(),
    )
    response = await agent.achat(data.query)
    return WhiskQueryBaseResponseSchema.from_llm_invoke(data.query, response.response)





📌 **What’s Different Here?**  
✅ **No Manual Agent Setup** – We **just define handlers** instead of manually wiring agents.  
✅ **Automatic Tool Selection** – `kitchenAI()` **automatically picks the right AI tool**.  
✅ **Scalability** – If we add **more AI functions**, we **don’t need to rewrite our code**.  

---

In [27]:

client = WhiskClient(
    nats_url="nats://nats.playground.kitchenai.dev:4222",
    client_id="1da00-a27a-400e-9568sdasd6uuu", #change to any random id you want. This has 1-1 relation with KtichenAI app. For playground, generate client_id from https://playground.kitchenai.dev/apps/playground/.
    user="playground", #optional if you are using playground url
    password="kitchenai_playground", #optional if you are using playground url
    kitchen=kitchen,
)
await client.run()

2025-02-14 20:48:40,733 INFO     -       |                                                                |            - Received
2025-02-14 20:48:40,733 INFO     -       |                                                                |            - Received
2025-02-14 20:48:40,733 INFO     -       |                                                                  |            - Received
2025-02-14 20:48:40,733 INFO     -       |                                                                  |            - Received
2025-02-14 20:48:40,733 INFO     -       |                                                                  |            - Received
2025-02-14 20:48:40,733 INFO     -       |                                                                  |            - Received
2025-02-14 20:48:40,733 INFO     -       |                                                               |            - Received
2025-02-14 20:48:40,738 INFO     -       |                                         

KitchenAI Configuration:

{
    'namespace': 'AI-Toolkit-Demo-2',
    'query_handlers': ['scraper', 'web-search'],
    'storage_handlers': [],
    'embed_handlers': [],
    'agent_handlers': []
}

✓ Successfully registered client!

2025-02-14 20:48:40,802 INFO     - FastStream app starting...


2025-02-14 20:48:40,884 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sdasd6uuu.query.*        |            - `HandleQuery` waiting for messages
2025-02-14 20:48:40,884 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sdasd6uuu.query.*        |            - `HandleQuery` waiting for messages
2025-02-14 20:48:40,884 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sdasd6uuu.query.*          |            - `HandleQuery` waiting for messages
2025-02-14 20:48:40,884 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sdasd6uuu.query.*          |            - `HandleQuery` waiting for messages
2025-02-14 20:48:40,884 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sdasd6uuu.query.*          |            - `HandleQuery` waiting for messages
2025-02-14 20:48:40,884 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sdasd6uuu.query.*          |            - `HandleQuery` waiting for messages
2025-02-14 20:48:40,884 INFO     - queue | kitchenai.ser

2025-02-14 20:48:40,906 INFO     - FastStream app started successfully! To exit, press CTRL+C


2025-02-14 20:50:16,090 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sdasd6uuu.query.*        | 0ef9b4c5-a - Received
2025-02-14 20:50:16,090 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sdasd6uuu.query.*        | 0ef9b4c5-a - Received
2025-02-14 20:50:16,090 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sdasd6uuu.query.*          | 0ef9b4c5-a - Received
2025-02-14 20:50:16,090 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sdasd6uuu.query.*          | 0ef9b4c5-a - Received
2025-02-14 20:50:16,090 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sdasd6uuu.query.*          | 0ef9b4c5-a - Received
2025-02-14 20:50:16,090 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sdasd6uuu.query.*          | 0ef9b4c5-a - Received
2025-02-14 20:50:16,090 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sdasd6uuu.query.*       | 0ef9b4c5-a - Received
2025-02-14 20:50:16,090 INFO     - queue | kitchenai.service.1da00-a27a-400e-9568sd

2025-02-14 21:00:03,582 INFO     - FastStream app shutting down...
2025-02-14 21:00:03,610 INFO     - FastStream app shut down gracefully.


📌 **Execution:**  
- The **Web Search Agent** handles search queries.  
- The **Scraper Agent** extracts data.  
- The **LLM processes everything** and returns a structured response.  
- **No need to manually call individual AI agents.**  

---

## What We Learned

1️⃣ **Traditional AI Workflows** require **manual agent setup, API calls, and tool management**.  
2️⃣ **KitchenAI** allows us to **register AI tools once and use them dynamically**.  
3️⃣ **Scalability** – We can **add new capabilities without rewriting code**.  
